# [Module 4.1] Deploy from Scratch

이 노트북은 스크립트 모드로 학습을 수행 합니다.
여기서는 다음과 같은 작업을 합니다.

- 학습할 데이타를 S3로 지정
- Train 학습 파리미터 설정
- Estimator를 생성하고 Train Docker Image 경로 설정
- train_instance_type 을 'ml.p3.2xlarge' 인스턴스로 설정
- S3에 생성된 모델 아티펙트 확인



필요한 프로그램 설치

In [1]:
!pip install -q --upgrade pip
!pip install -q wrapt --upgrade --ignore-installed
!pip install -q tensorflow==2.1.0
!pip install -q transformers==2.8.0
!pip install -q sagemaker==1.56.1

ERROR: astroid 2.3.3 has requirement wrapt==1.11.*, but you'll have wrapt 1.12.1 which is incompatible.


In [5]:
import boto3
import sagemaker
import pandas as pd
import tensorflow as tf
import os

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

## Download the Model to the Notebook

In [11]:
%store -r training_job_name

In [12]:
print(training_job_name)

bert2tweet-2020-07-22-11-41-39-445


In [13]:
model_download = 'model'
os.makedirs(model_download, exist_ok=True)

In [14]:
!aws s3 cp s3://$bucket/$training_job_name/output/model.tar.gz {model_download}/model.tar.gz

download: s3://sagemaker-us-west-2-057716757052/bert2tweet-2020-07-22-11-41-39-445/output/model.tar.gz to model/model.tar.gz


In [37]:
!tar -xvzf   {model_download}/model.tar.gz
!saved_model_cli show --all --dir ./tensorflow/saved_model/0/

tensorflow/
tensorflow/saved_model/
tensorflow/saved_model/0/
tensorflow/saved_model/0/saved_model.pb
tensorflow/saved_model/0/variables/
tensorflow/saved_model/0/variables/variables.data-00000-of-00002
tensorflow/saved_model/0/variables/variables.data-00001-of-00002
tensorflow/saved_model/0/variables/variables.index
tensorflow/saved_model/0/assets/
transformers/
transformers/fine-tuned/
transformers/fine-tuned/config.json
transformers/fine-tuned/tf_model.h5
tensorflow/
tensorflow/saved_model/
tensorflow/saved_model/0/
tensorflow/saved_model/0/saved_model.pb
tensorflow/saved_model/0/variables/
tensorflow/saved_model/0/variables/variables.index
tensorflow/saved_model/0/variables/variables.data-00000-of-00002
tensorflow/saved_model/0/variables/variables.data-00001-of-00002
tensorflow/saved_model/0/assets/
transformers/
transformers/fine-tuned/
transformers/fine-tuned/config.json
transformers/fine-tuned/tf_model.h5
2020-07-22 12:13:20.446099: W tensorflow/stream_executor/platform/default/

## SageMaker Model 생성

In [31]:
import os
from sagemaker.tensorflow.serving import Model

model = Model(model_data='s3://{}/{}/output/model.tar.gz'.format(bucket, training_job_name),
              role=role,
              framework_version='2.0.0') # Elastic Inference does not yet support TF 2.1.0 as of sagemaker==1.56.1

## Endpoint 생성

In [32]:
instance_type='ml.m4.xlarge'

deployed_model = model.deploy(initial_instance_count = 1,
                             instance_type = instance_type,
                             wait=True)

endpoint_name = deployed_model.endpoint
print('Endpoint name:  {}'.format(endpoint_name))

-------------!Endpoint name:  tensorflow-inference-2020-07-22-12-06-28-165


## Inference Request Serializer and Deserializer 생성

In [6]:
class RequestHandler(object):
    import json
    
    def __init__(self, tokenizer, max_seq_length):
        self.tokenizer = tokenizer
        self.max_seq_length = max_seq_length

    def __call__(self, instances):
        transformed_instances = []

        for instance in instances:
            encode_plus_tokens = tokenizer.encode_plus(instance,
                                                       pad_to_max_length=True,
                                                       max_length=self.max_seq_length)

            input_ids = encode_plus_tokens['input_ids']
            input_mask = encode_plus_tokens['attention_mask']
            segment_ids = [0] * self.max_seq_length

            transformed_instance = {"input_ids": input_ids, 
                                    "input_mask": input_mask, 
                                    "segment_ids": segment_ids}

            transformed_instances.append(transformed_instance)

        transformed_data = {"instances": transformed_instances}

        return json.dumps(transformed_data)
    
class ResponseHandler(object):
    import json
    import tensorflow as tf
    
    def __init__(self, classes):
        self.classes = classes
    
    def __call__(self, response, accept_header):
        import tensorflow as tf

        response_body = response.read().decode('utf-8')

        response_json = json.loads(response_body)

        log_probabilities = response_json["predictions"]

        predicted_classes = []

        # Convert log_probabilities => softmax (all probabilities add up to 1) => argmax (final prediction)
        for log_probability in log_probabilities:
            softmax = tf.nn.softmax(log_probability)    
            predicted_class_idx = tf.argmax(softmax, axis=-1, output_type=tf.int32)
            predicted_class = self.classes[predicted_class_idx]
            predicted_classes.append(predicted_class)

        return predicted_classes    

## Predictor 생성

In [7]:
endpoint_name = 'tensorflow-inference-2020-07-22-12-06-28-165'

In [8]:
import json
from sagemaker.tensorflow.serving import Predictor
from transformers import DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

request_handler = RequestHandler(tokenizer=tokenizer,
                                 max_seq_length=128)

response_handler = ResponseHandler(classes=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

predictor = Predictor(endpoint_name=endpoint_name,
                      sagemaker_session=sess,
                      serializer=request_handler,
                      deserializer=response_handler,
                      content_type='application/json',
                      model_name='saved_model',
                      model_version=0)

## Inference 실행

In [9]:
test_file_path = 'data/test/tweet_file_test.csv'
test_df = pd.read_csv(test_file_path)


In [10]:
test_file_path = 'data/test/tweet_file_test.csv'
test_df = pd.read_csv(test_file_path)
test_df.TWEET.values

array(['@theweeknd meadows fest in the day and snl at night ny gonna be',
       "@aye_mickey sadiebabyy_ oh gosh i just hope y'all don't act up too much",
       'me af', ...,
       '@gingerjamie222 baby pull me closer in the backseat of your corsa. rt and like tag someone it applies to full cover on my faceboo',
       '@tanyawarren handmade to order pretty gift for a new baby flockbn kprs eshopsuk mummybloggers https://',
       'real life'], dtype=object)

In [11]:
sample_df = test_df.sample(10)
sample_df.columns

Index(['TWEET', 'LABEL'], dtype='object')

In [12]:
from TweetUtil import TweetUtil

tweet_util = TweetUtil()
tweet_util.load_emoji_data('emoji_to_idx.pickle')
emoji = tweet_util.get_emo_class_label(3)
print(emoji)

emoji_to_idx is loaded
😂


In [17]:
import tensorflow as tf
import json

sample_df = test_df.sample(10)
columns = ['TWEET', 'LABEL']
for tweet, label in zip(sample_df.TWEET.values, sample_df.LABEL.values):
    # print("label: {}, tweet: {}".format(label, tweet))
    
    reviews = [tweet]

    predicted_classes = predictor.predict(reviews)[0]
    print('Prediction, Ground_truth- {}:{}, {}:{} \n tweet: {}'.format(
        predicted_classes, 
        tweet_util.get_emo_class_label(predicted_classes),
        label, 
        tweet_util.get_emo_class_label(label),        
        tweet))    

        



Prediction, Ground_truth- 3:😂, 4:😊 
 tweet: happy birthday bud ryanromppainen
Prediction, Ground_truth- 3:😂, 3:😂 
 tweet: @ghetto chief keef freestyle be like
Prediction, Ground_truth- 9:🤔, 1:💕 
 tweet: @kentkristensen1 nature spring coming take walks with loved ones relax no stress find love again and again romance and happiness ht
Prediction, Ground_truth- 3:😂, 3:😂 
 tweet: @majforevs conference call with these two frvrmjrss lovekonicoleee oplanticketbuying
Prediction, Ground_truth- 3:😂, 7:😭 
 tweet: @itskerrii this is exactly how it happened
Prediction, Ground_truth- 3:😂, 8:🙄 
 tweet: i've watched not just 1 but 2 shows in a row i recorded without skipping the commercials
Prediction, Ground_truth- 3:😂, 7:😭 
 tweet: @dolo_kd non-jamaicans can't tell me how to eat jamaican food are you dumb
Prediction, Ground_truth- 3:😂, 2:🔥 
 tweet: hey daftslag you may want to check for latest updates on creamfields _
Prediction, Ground_truth- 3:😂, 3:😂 
 tweet: this is so me!
Prediction, Ground_trut

In [36]:
    
reviews = ["This is great!", 
           "This is not good."]

predicted_classes = predictor.predict(reviews)

for predicted_class, review in zip(predicted_classes, reviews):
    print('[Predicted Star Rating: {}]'.format(predicted_class), review)

[Predicted Star Rating: 3] This is great!
[Predicted Star Rating: 3] This is not good.
